# Note4_統整資料並製作出最終表來給R作圖

# 1. 整理 aed 與 edc 的 loose positive MAGs 資料

In [5]:
import os
import pandas as pd
import re

# Read table
aed_final_df = pd.read_csv('../data/processed/Final/Actino/aed_PositiveHits_ForR_loose.csv')
edc_final_df = pd.read_csv('../data/processed/Final/Proteo/edc_PositiveHits_ForR_loose.csv')

# concatenate aed and edc final data
B50_HMMFinal_df = pd.concat([aed_final_df, edc_final_df])

# remove duplicated and update the 'Both_clsuter' in homologous_cluster column
# 找重複 (有2)
B50_duplicated = B50_HMMFinal_df['genome_id'].value_counts()

# set new column in DataFrame with value counts
B50_HMMFinal_df['num_duplicated'] = B50_HMMFinal_df['genome_id'].map(B50_duplicated)

# if number is 2 (duplicated) change the homnologous column to 'Both_cluster'
B50_HMMFinal_df.loc[B50_HMMFinal_df['num_duplicated'] == 2 , 'Homologous_cluster'] = 'Both_cluster'

# remove duplicated row
B50_HMMFinal_df = B50_HMMFinal_df.drop_duplicates(subset=['genome_id'])
print('B50_final Row (1):', len(B50_HMMFinal_df.index))

# remove, add and rearrange column
B50_HMMFinal_df.drop('num_duplicated', axis=1, inplace=True)
B50_HMMFinal_df = B50_HMMFinal_df.iloc[:, 1:]

# open file for vlookup from phylum to final taxonomy ID
PhylumToTaxonomy = pd.read_csv('../data/interim/PhylumToTaxonomy.csv')
ClassToTaxnomy = pd.read_csv('../data/interim/ClassToTaxonomy.csv')

# merge with phylum column
B50_HMMFinal_df1 = pd.merge(B50_HMMFinal_df, PhylumToTaxonomy, on='Phylum', how='left')

# merge with Class
B50_HMMFinal_df2 = pd.merge(B50_HMMFinal_df, ClassToTaxnomy, on='Class', how='left')

# concatenate the two dataframes
B50_HMMFinal_df = pd.concat([B50_HMMFinal_df1, B50_HMMFinal_df2], axis=0, ignore_index=True)

# drop rows with NaN values in taxonomy column
B50_HMMFinal_df = B50_HMMFinal_df.dropna(subset=['taxonomy'])
print('B50_final Row (2):', len(B50_HMMFinal_df.index))

# remove and rerrange column
B50_HMMFinal_df.drop(columns=['Phylum', 'Class'], inplace=True)
B50_HMMFinal_df = B50_HMMFinal_df.reindex(columns=['genome_id', 'num_hits', 'taxonomy', 'ecosystem', 'ecosystem_category', 'Homologous_cluster', 'longitude', 'latitude'])
B50_HMMFinal_df.to_csv('../data/processed/Final/Combined/Combined_PositiveHits_loose.csv')
print('Any Null in B50_HMMFinal_df:\n', B50_HMMFinal_df.isnull().any())

# calculate hits number
B50_HMMFinal_number = B50_HMMFinal_df.groupby(['ecosystem', 'ecosystem_category', 'taxonomy']).size().reset_index(name='NumberOfMAGs')
B50_HMMFinal_number.to_csv('../data/processed/Final/R/B50_HMMFinal_loose_ForBarChart.csv', index=False)

print(B50_HMMFinal_number['taxonomy'].unique())
print(B50_HMMFinal_number['ecosystem'].unique())
print(B50_HMMFinal_number['ecosystem_category'].unique())
display(B50_HMMFinal_number)

# mmerge reference datasheet to get a full table for readers and Output

# read files
genome_metadata = pd.read_csv('../data/external/Reference_Data/genome_metadata.csv')
genmoe_metadata_edit = pd.read_csv('../data/external/Reference_Data/genome_metadata_edit.csv')
genome_metadata_editForAnalysis_NotReference = pd.read_csv('../data/external/genome_metadata_editForAnalysis_NotReference.csv')

# Got the specific columns in B50_HMMFinal_df
B50_HMMFinal_df_HH = B50_HMMFinal_df[['genome_id', 'num_hits', 'Homologous_cluster']]

# merge table
M_genome_metadata = pd.merge(B50_HMMFinal_df_HH, genome_metadata, on='genome_id', how='left')
M_genmoe_metadata_edit = pd.merge(B50_HMMFinal_df_HH, genmoe_metadata_edit, on='genome_id', how='left')
M_genome_metadata_editForAnalysis_NotReference = pd.merge(B50_HMMFinal_df_HH, genome_metadata_editForAnalysis_NotReference, on='genome_id', how='left')

# save files
M_genome_metadata.to_csv('../data/processed/Final/ForReader/PositiveHits_loose_genome_metadata.csv', index=False)
M_genmoe_metadata_edit.to_csv('../data/processed/Final/ForReader/PositiveHits_loose_genmoe_metadata_edit.csv', index=False)
M_genome_metadata_editForAnalysis_NotReference.to_csv('../data/processed/Final/ForReader/PositiveHits_loose_genome_metadata_editForAnalysis_NotReference.csv', index=False)

# done
print('done')

B50_final Row (1): 609
B50_final Row (2): 609
Any Null in B50_HMMFinal_df:
 genome_id             False
num_hits              False
taxonomy              False
ecosystem             False
ecosystem_category    False
Homologous_cluster    False
longitude              True
latitude               True
dtype: bool
['Actinobacteria' 'Alphaproteobacteria' 'Chloroflexi'
 'Gammaproteobacteria' 'Myxococcia' 'Thermodesulfobacterota'
 'Unclassified Bacteria' 'Other Bacterial Phylum']
['Aquatic' 'Engineered' 'Host-associated' 'Terrestrial']
['Freshwater' 'Marine' 'Non-marine Saline and Alkaline' 'Sediment'
 'Thermal springs' 'Bioreactor' 'Built environment' 'Lab enrichment'
 'Lab synthesis' 'Modeled' 'Solid waste' 'Wastewater' 'Annelida' 'Fungi'
 'Invertebrates' 'Mammals' 'Microbial' 'Plants' 'Porifera'
 'Deep subsurface' 'Peat' 'Soil']


,ecosystem,ecosystem_category,taxonomy,NumberOfMAGs
0,Aquatic,Freshwater,Actinobacteria,52
1,Aquatic,Freshwater,Alphaproteobacteria,17
2,Aquatic,Freshwater,Chloroflexi,8
3,Aquatic,Freshwater,Gammaproteobacteria,12
4,Aquatic,Freshwater,Myxococcia,1
...,...,...,...,...
57,Terrestrial,Soil,Alphaproteobacteria,10
58,Terrestrial,Soil,Chloroflexi,2
59,Terrestrial,Soil,Gammaproteobacteria,5
60,Terrestrial,Soil,Other Bacterial Phylum,1


done


# 2. loose hits distribution of world Map

In [6]:
import os
import pandas as pd
import re

# Read table
aed_final_df = pd.read_csv('../data/processed/Final/Actino/aed_PositiveHits_ForR_loose.csv')
edc_final_df = pd.read_csv('../data/processed/Final/Proteo/edc_PositiveHits_ForR_loose.csv')

# concatenate aed and edc final data
B50_HMMFinal_df = pd.concat([aed_final_df, edc_final_df])

# remove duplicated and update the 'Both_clsuter' in homologous_cluster column
# 找重複 (有2)
B50_duplicated = B50_HMMFinal_df['genome_id'].value_counts()

# set new column in DataFrame with value counts
B50_HMMFinal_df['num_duplicated'] = B50_HMMFinal_df['genome_id'].map(B50_duplicated)

# if number is 2 (duplicated) change the homnologous column to 'Both_cluster'
B50_HMMFinal_df.loc[B50_HMMFinal_df['num_duplicated'] == 2 , 'Homologous_cluster'] = 'Both_cluster'

# remove duplicated row
B50_HMMFinal_df = B50_HMMFinal_df.drop_duplicates(subset=['genome_id'])
print('B50_final Row (1):', len(B50_HMMFinal_df.index))

# remove, add and rearrange column
B50_HMMFinal_df.drop('num_duplicated', axis=1, inplace=True)
B50_HMMFinal_df = B50_HMMFinal_df.iloc[:, 1:]

# open file for vlookup from phylum to final taxonomy ID
PhylumToTaxonomy = pd.read_csv('../data/interim/PhylumToTaxonomy.csv')
ClassToTaxnomy = pd.read_csv('../data/interim/ClassToTaxonomy.csv')

# merge with phylum column
B50_HMMFinal_df1 = pd.merge(B50_HMMFinal_df, PhylumToTaxonomy, on='Phylum', how='left')

# merge with Class
B50_HMMFinal_df2 = pd.merge(B50_HMMFinal_df, ClassToTaxnomy, on='Class', how='left')

# concatenate the two dataframes
B50_HMMFinal_df = pd.concat([B50_HMMFinal_df1, B50_HMMFinal_df2], axis=0, ignore_index=True)

# drop rows with NaN values in taxonomy column
B50_HMMFinal_df = B50_HMMFinal_df.dropna(subset=['taxonomy'])
print('B50_final Row (2):', len(B50_HMMFinal_df.index))

# remove and rerrange column
B50_HMMFinal_df.drop(columns=['Phylum', 'Class', 'num_hits', 'taxonomy', 'Homologous_cluster'], inplace=True)
B50_HMMFinal_df = B50_HMMFinal_df.reindex(columns=['genome_id', 'metagenome_id', 'ecosystem', 'ecosystem_category', 'longitude', 'latitude'])

# check missing row of coordinates
missing_rows = B50_HMMFinal_df[B50_HMMFinal_df[['longitude', 'latitude']].isnull().any(axis=1)]
print(f'There are {missing_rows.shape[0]} row(s) with null values.')

# Got the metagenomic_id with missing coordinate
print('metagenomic_id with missing coordinate: \n',missing_rows['metagenome_id'].unique )

# remove unnecessary columns
B50_HMMFinal_df = B50_HMMFinal_df.drop(['genome_id', 'metagenome_id'], axis=1)

# rename columns
B50_HMMFinal_df = B50_HMMFinal_df.rename(columns={'ecosystem': 'Ecosystem', 'ecosystem_category': 'Ecosystem_Category', 'longitude': 'Longitude', 'latitude': 'Latitude'})

# reset index and write files
B50_HMMFinal_df = B50_HMMFinal_df.reset_index(drop=True)

# assuming your data is in a DataFrame called df
B50_HMMFinal_df_grouped = B50_HMMFinal_df.groupby(['Ecosystem', 'Ecosystem_Category', 'Latitude', 'Longitude']).size().reset_index(name='count')
B50_HMMFinal_df_grouped.to_csv('../data/processed/Final/R/B50_HMMFinal_loose_ForMaps.csv', index=False)

print('Any Null in edc_Positive_metagenomes_df:\n', B50_HMMFinal_df_grouped.isnull().any())
print('done')
B50_HMMFinal_df_grouped


B50_final Row (1): 609
B50_final Row (2): 609
There are 57 row(s) with null values.
metagenomic_id with missing coordinate: 
 <bound method Series.unique of 17      3300026210
46      3300012014
57      3300026262
58      3300026200
59      3300026199
61      3300026202
63      3300026206
64      3300026261
67      3300026256
68      3300026193
69      3300026209
89      3300026212
92      3300012005
120     3300026267
128     3300026265
131     3300026108
141     3300026262
142     3300026192
144     3300026262
149     3300026092
161     3300026091
215     3300027855
235     3300026213
237     3300026211
238     3300026208
341     3300026079
345     3300026087
346     3300026188
349     3300026092
355     3300026108
358     3300026093
360     3300026119
365     3300006902
381     3300026200
424     3300026279
425     3300026269
426     3300026268
427     3300026266
429     3300026265
431     3300026264
432     3300026263
433     3300026260
434     3300026260
435     3300026260
436    

,Ecosystem,Ecosystem_Category,Latitude,Longitude,count
0,Aquatic,Freshwater,-78.0836,164.284,1
1,Aquatic,Freshwater,-78.0544,164.021,1
2,Aquatic,Freshwater,-78.0482,164.113,1
3,Aquatic,Freshwater,-78.0267,163.820,1
4,Aquatic,Freshwater,-77.7140,162.445,8
...,...,...,...,...,...
225,Terrestrial,Soil,68.6139,-149.314,1
226,Terrestrial,Soil,71.2838,-156.598,2
227,Terrestrial,Soil,71.2906,-156.789,1
228,Terrestrial,Soil,71.2990,-156.610,2


## 不使用了:整理 aed 與 edc 的 critical positive MAGs 資料

In [1]:
# Read table
import os
import pandas as pd
import re

aed_final_df = pd.read_csv('../data/processed/Final/Actino/aed_PositiveHits_ForR_critical.csv')
edc_final_df = pd.read_csv('../data/processed/Final/Proteo/edc_PositiveHits_ForR_critical.csv')

# concatenate aed and edc final data
B50_HMMFinal_df = pd.concat([aed_final_df, edc_final_df])

# remove duplicated and update the 'Both_clsuter' in homologous_cluster column
# 找重複 (有2)
B50_duplicated = B50_HMMFinal_df['genome_id'].value_counts()

# set new column in DataFrame with value counts
B50_HMMFinal_df['num_duplicated'] = B50_HMMFinal_df['genome_id'].map(B50_duplicated)

# if number is 2 (duplicated) change the homnologous column to 'Both_cluster'
B50_HMMFinal_df.loc[B50_HMMFinal_df['num_duplicated'] == 2 , 'Homologous_cluster'] = 'Both_cluster'

# remove duplicated row
B50_HMMFinal_df = B50_HMMFinal_df.drop_duplicates(subset=['genome_id'])
print('B50_final Row (1):', len(B50_HMMFinal_df.index))

# remove, add and rearrange column
B50_HMMFinal_df.drop('num_duplicated', axis=1, inplace=True)
B50_HMMFinal_df = B50_HMMFinal_df.iloc[:, 1:]

# open file for vlookup from phylum to final taxonomy ID
PhylumToTaxonomy = pd.read_csv('../data/interim/PhylumToTaxonomy.csv')
ClassToTaxnomy = pd.read_csv('../data/interim/ClassToTaxonomy.csv')

# merge with phylum column
B50_HMMFinal_df1 = pd.merge(B50_HMMFinal_df, PhylumToTaxonomy, on='Phylum', how='left')


# merge with Class
B50_HMMFinal_df2 = pd.merge(B50_HMMFinal_df, ClassToTaxnomy, on='Class', how='left')

# concatenate the two dataframes
B50_HMMFinal_df = pd.concat([B50_HMMFinal_df1, B50_HMMFinal_df2], axis=0, ignore_index=True)

# drop rows with NaN values in taxonomy column
B50_HMMFinal_df = B50_HMMFinal_df.dropna(subset=['taxonomy'])
print('B50_final Row (2):', len(B50_HMMFinal_df.index))

# remove and rerrange column
B50_HMMFinal_df.drop(columns=['Phylum', 'Class'], inplace=True)
B50_HMMFinal_df = B50_HMMFinal_df.reindex(columns=['genome_id', 'num_hits', 'taxonomy', 'ecosystem', 'ecosystem_category', 'Homologous_cluster', 'longitude', 'latitude'])
B50_HMMFinal_df.to_csv('../data/processed/Final/Combined/Combined_PositiveHits_critical.csv')
print('Any Null in B50_HMMFinal_df:\n', B50_HMMFinal_df.isnull().any())

# calculate hits number
B50_HMMFinal_number = B50_HMMFinal_df.groupby(['ecosystem', 'ecosystem_category', 'taxonomy']).size().reset_index(name='NumberOfMAGs')
B50_HMMFinal_number.to_csv('../data/processed/Final/R/B50_HMMFinal_critical_ForBarChart.csv', index=False)

print(B50_HMMFinal_number['taxonomy'].unique())
print(B50_HMMFinal_number['ecosystem'].unique())
print(B50_HMMFinal_number['ecosystem_category'].unique())
display(B50_HMMFinal_number)

# mmerge reference datasheet to get a full table for readers and Output

# read files
genome_metadata = pd.read_csv('../data/external/Reference_Data/genome_metadata.csv')
genmoe_metadata_edit = pd.read_csv('../data/external/Reference_Data/genome_metadata_edit.csv')
genome_metadata_editForAnalysis_NotReference = pd.read_csv('../data/external/genome_metadata_editForAnalysis_NotReference.csv')

# Got the specific columns in B50_HMMFinal_df
B50_HMMFinal_df_HH = B50_HMMFinal_df[['genome_id', 'num_hits', 'Homologous_cluster']]

# merge table
M_genome_metadata = pd.merge(B50_HMMFinal_df_HH, genome_metadata, on='genome_id', how='left')
M_genmoe_metadata_edit = pd.merge(B50_HMMFinal_df_HH, genmoe_metadata_edit, on='genome_id', how='left')
M_genome_metadata_editForAnalysis_NotReference = pd.merge(B50_HMMFinal_df_HH, genome_metadata_editForAnalysis_NotReference, on='genome_id', how='left')

# save files
M_genome_metadata.to_csv('../data/processed/Final/ForReader/PositiveHits_critical_genome_metadata.csv', index=False)
M_genmoe_metadata_edit.to_csv('../data/processed/Final/ForReader/PositiveHits_critical_genmoe_metadata_edit.csv', index=False)
M_genome_metadata_editForAnalysis_NotReference.to_csv('../data/processed/Final/ForReader/PositiveHits_critical_genome_metadata_editForAnalysis_NotReference.csv', index=False)

# done
print('done')

B50_final Row (1): 35
B50_final Row (2): 35
Any Null in B50_HMMFinal_df:
 genome_id             False
num_hits              False
taxonomy              False
ecosystem             False
ecosystem_category    False
Homologous_cluster    False
longitude             False
latitude              False
dtype: bool
['p_Actinobacteriota_c_Acidimicrobiia'
 'p_Proteobacteria_c_Alphaproteobacteria' 'p_Myxococcota'
 'p_Proteobacteria_c_Gammaproteobacteria']
['Aquatic' 'Engineered' 'Host-associated' 'Terrestrial']
['Freshwater' 'Marine' 'Thermal springs' 'Wastewater' 'Annelida' 'Fungi'
 'Plants' 'Soil']


,ecosystem,ecosystem_category,taxonomy,NumberOfMAGs
0,Aquatic,Freshwater,p_Actinobacteriota_c_Acidimicrobiia,1
1,Aquatic,Freshwater,p_Proteobacteria_c_Alphaproteobacteria,3
2,Aquatic,Marine,p_Myxococcota,3
3,Aquatic,Marine,p_Proteobacteria_c_Alphaproteobacteria,1
4,Aquatic,Marine,p_Proteobacteria_c_Gammaproteobacteria,16
5,Aquatic,Thermal springs,p_Myxococcota,1
6,Engineered,Wastewater,p_Proteobacteria_c_Gammaproteobacteria,1
7,Host-associated,Annelida,p_Proteobacteria_c_Gammaproteobacteria,1
8,Host-associated,Fungi,p_Proteobacteria_c_Alphaproteobacteria,2
9,Host-associated,Plants,p_Proteobacteria_c_Alphaproteobacteria,2


done


# 不執行

## 3. 整理 aed 與 edc 的 10 Hits positive MAGs 資料

In [1]:
# Read table
import os
import pandas as pd
import re

aed_final_df = pd.read_csv('../data/processed/Final/Actino/aed_PositiveHits_ForR_10Hits.csv')
edc_final_df = pd.read_csv('../data/processed/Final/Proteo/edc_PositiveHits_ForR_10Hits.csv')

# concatenate aed and edc final data
B50_HMMFinal_df = pd.concat([aed_final_df, edc_final_df])

# remove duplicated and update the 'Both_clsuter' in homologous_cluster column
# 找重複 (有2)
B50_duplicated = B50_HMMFinal_df['genome_id'].value_counts()

# set new column in DataFrame with value counts
B50_HMMFinal_df['num_duplicated'] = B50_HMMFinal_df['genome_id'].map(B50_duplicated)

# if number is 2 (duplicated) change the homnologous column to 'Both_cluster'
B50_HMMFinal_df.loc[B50_HMMFinal_df['num_duplicated'] == 2 , 'Homologous_cluster'] = 'Both_cluster'

# remove duplicated row
B50_HMMFinal_df = B50_HMMFinal_df.drop_duplicates(subset=['genome_id'])
print('B50_final Row (1):', len(B50_HMMFinal_df.index))

# remove, add and rearrange column
B50_HMMFinal_df.drop('num_duplicated', axis=1, inplace=True)
B50_HMMFinal_df = B50_HMMFinal_df.iloc[:, 1:]

# open file for vlookup from phylum to final taxonomy ID
PhylumToTaxonomy = pd.read_csv('../data/interim/PhylumToTaxonomy.csv')
ClassToTaxnomy = pd.read_csv('../data/interim/ClassToTaxonomy.csv')

# merge with phylum column
B50_HMMFinal_df1 = pd.merge(B50_HMMFinal_df, PhylumToTaxonomy, on='Phylum', how='left')


# merge with Class
B50_HMMFinal_df2 = pd.merge(B50_HMMFinal_df, ClassToTaxnomy, on='Class', how='left')

# concatenate the two dataframes
B50_HMMFinal_df = pd.concat([B50_HMMFinal_df1, B50_HMMFinal_df2], axis=0, ignore_index=True)

# drop rows with NaN values in taxonomy column
B50_HMMFinal_df = B50_HMMFinal_df.dropna(subset=['taxonomy'])
print('B50_final Row (2):', len(B50_HMMFinal_df.index))
B50_HMMFinal_df
B50_HMMFinal_df.to_csv('../data/processed/Final/Combined/Combined_PositiveHits_10Hits.csv')

# remove and rerrange column
B50_HMMFinal_df.drop(columns=['Phylum', 'Class'], inplace=True)
B50_HMMFinal_df = B50_HMMFinal_df.reindex(columns=['genome_id', 'num_hits', 'taxonomy', 'ecosystem', 'ecosystem_type', 'Homologous_cluster'])

# calculate hits number
B50_HMMFinal_number = B50_HMMFinal_df.groupby(['ecosystem', 'ecosystem_type', 'taxonomy']).size().reset_index(name='NumberOfMAGs')
B50_HMMFinal_number.to_csv('../data/processed/Final/R/B50_HMMFinal_10Hits_ForBarChart.csv', index=False)


# done
print(B50_HMMFinal_number['taxonomy'].unique())
print(B50_HMMFinal_number['ecosystem'].unique())
print(B50_HMMFinal_number['ecosystem_type'].unique())
print('done')
B50_HMMFinal_number

B50_final Row (1): 114
B50_final Row (2): 114
['p_Actinobacteriota_c_Acidimicrobiia' 'p_Myxococcota'
 'p_Proteobacteria_c_Gammaproteobacteria'
 'p_Proteobacteria_c_Alphaproteobacteria' 'p_Unclassified Bacteria'
 'p_Actinobacteriota_c_Actinobacteria']
['Aquatic' 'Engineered' 'Host-associated' 'Terrestrial']
['Freshwater' 'Marine' 'Sediment' 'Thermal spring' 'Lab enrichment'
 'Wastewater' 'Annelida' 'Cnidaria' 'Fungi' 'Plant' 'Soil']
done


,ecosystem,ecosystem_type,taxonomy,NumberOfMAGs
0,Aquatic,Freshwater,p_Actinobacteriota_c_Acidimicrobiia,15
1,Aquatic,Freshwater,p_Myxococcota,1
2,Aquatic,Freshwater,p_Proteobacteria_c_Gammaproteobacteria,2
3,Aquatic,Marine,p_Actinobacteriota_c_Acidimicrobiia,31
4,Aquatic,Marine,p_Myxococcota,10
5,Aquatic,Marine,p_Proteobacteria_c_Alphaproteobacteria,1
6,Aquatic,Marine,p_Proteobacteria_c_Gammaproteobacteria,9
7,Aquatic,Marine,p_Unclassified Bacteria,4
8,Aquatic,Sediment,p_Actinobacteriota_c_Actinobacteria,1
9,Aquatic,Thermal spring,p_Actinobacteriota_c_Acidimicrobiia,1


## 4. 整理 aed 與 edc 的 reference positive MAGs 資料

In [1]:
# Read table
import os
import pandas as pd
import re

aed_final_df = pd.read_csv('../data/processed/Final/Actino/aed_PositiveHits_ForR_reference.csv')
edc_final_df = pd.read_csv('../data/processed/Final/Proteo/edc_PositiveHits_ForR_reference.csv')

# concatenate aed and edc final data
B50_HMMFinal_df = pd.concat([aed_final_df, edc_final_df])

# remove duplicated and update the 'Both_clsuter' in homologous_cluster column
# 找重複 (有2)
B50_duplicated = B50_HMMFinal_df['genome_id'].value_counts()

# set new column in DataFrame with value counts
B50_HMMFinal_df['num_duplicated'] = B50_HMMFinal_df['genome_id'].map(B50_duplicated)

# if number is 2 (duplicated) change the homnologous column to 'Both_cluster'
B50_HMMFinal_df.loc[B50_HMMFinal_df['num_duplicated'] == 2 , 'Homologous_cluster'] = 'Both_cluster'

# remove duplicated row
B50_HMMFinal_df = B50_HMMFinal_df.drop_duplicates(subset=['genome_id'])
print('B50_final Row (1):', len(B50_HMMFinal_df.index))

# remove, add and rearrange column
B50_HMMFinal_df.drop('num_duplicated', axis=1, inplace=True)
B50_HMMFinal_df = B50_HMMFinal_df.iloc[:, 1:]

# open file for vlookup from phylum to final taxonomy ID
PhylumToTaxonomy = pd.read_csv('../data/interim/PhylumToTaxonomy.csv')
ClassToTaxnomy = pd.read_csv('../data/interim/ClassToTaxonomy.csv')

# merge with phylum column
B50_HMMFinal_df1 = pd.merge(B50_HMMFinal_df, PhylumToTaxonomy, on='Phylum', how='left')


# merge with Class
B50_HMMFinal_df2 = pd.merge(B50_HMMFinal_df, ClassToTaxnomy, on='Class', how='left')

# concatenate the two dataframes
B50_HMMFinal_df = pd.concat([B50_HMMFinal_df1, B50_HMMFinal_df2], axis=0, ignore_index=True)

# drop rows with NaN values in taxonomy column
B50_HMMFinal_df = B50_HMMFinal_df.dropna(subset=['taxonomy'])
print('B50_final Row (2):', len(B50_HMMFinal_df.index))
B50_HMMFinal_df
B50_HMMFinal_df.to_csv('../data/processed/Final/Combined/Combined_PositiveHits_reference.csv')

# remove and rerrange column
B50_HMMFinal_df.drop(columns=['Phylum', 'Class'], inplace=True)
B50_HMMFinal_df = B50_HMMFinal_df.reindex(columns=['genome_id', 'num_hits', 'taxonomy', 'ecosystem', 'ecosystem_type', 'Homologous_cluster'])

# calculate hits number
B50_HMMFinal_number = B50_HMMFinal_df.groupby(['ecosystem', 'ecosystem_type', 'taxonomy']).size().reset_index(name='NumberOfMAGs')
B50_HMMFinal_number.to_csv('../data/processed/Final/R/B50_HMMFinal_reference_ForBarChart.csv', index=False)


# done
print(B50_HMMFinal_number['taxonomy'].unique())
print(B50_HMMFinal_number['ecosystem'].unique())
print(B50_HMMFinal_number['ecosystem_type'].unique())
print('done')
B50_HMMFinal_number

B50_final Row (1): 315
B50_final Row (2): 315
['p_Actinobacteriota_c_Acidimicrobiia'
 'p_Actinobacteriota_c_Actinobacteria' 'p_Myxococcota'
 'p_Proteobacteria_c_Alphaproteobacteria'
 'p_Proteobacteria_c_Gammaproteobacteria'
 'p_Chloroflexota_c_Dehalococcoidia' 'p_Other bacterial phylum']
['Aquatic' 'Engineered' 'Host-associated' 'Terrestrial']
['Freshwater' 'Marine' 'Non-marine Saline and Alkaline' 'Sediment'
 'Thermal spring' 'Bioreactor' 'Bioremediation' 'Built environment'
 'Lab enrichment' 'Solid waste' 'Wastewater' 'Algae' 'Annelida'
 'Arthropoda' 'Cnidaria' 'Fungi' 'Insecta' 'Mammal' 'Plant' 'Porifera'
 'Deep subsurface' 'Soil']
done


,ecosystem,ecosystem_type,taxonomy,NumberOfMAGs
0,Aquatic,Freshwater,p_Actinobacteriota_c_Acidimicrobiia,5
1,Aquatic,Freshwater,p_Actinobacteriota_c_Actinobacteria,4
2,Aquatic,Freshwater,p_Myxococcota,1
3,Aquatic,Freshwater,p_Proteobacteria_c_Alphaproteobacteria,17
4,Aquatic,Freshwater,p_Proteobacteria_c_Gammaproteobacteria,8
5,Aquatic,Marine,p_Actinobacteriota_c_Acidimicrobiia,8
6,Aquatic,Marine,p_Actinobacteriota_c_Actinobacteria,15
7,Aquatic,Marine,p_Chloroflexota_c_Dehalococcoidia,1
8,Aquatic,Marine,p_Myxococcota,3
9,Aquatic,Marine,p_Proteobacteria_c_Alphaproteobacteria,28


# 補充

In [47]:
# # create a sample dataframe
# import pandas as pd
# data = {'A': ['foo', 'bar', 'baz', 'foo', 'bar', 'baz'], 'B': [1, 2, 3, 4, 5, 6]}
# df = pd.DataFrame(data)

# # remove duplicated rows and update values in column 'B'
# mask = df.duplicated(subset=['A'])
# df.loc[mask, 'B'] = 999
# df = df[~mask]
# df

,A,B
0,foo,1
1,bar,2
2,baz,3
